In [196]:
from sys import prefix

import pandas as pd
import pandas_ta as ta
import yfinance as yf
from html5lib.constants import prefixes

In [197]:
org_data = yf.download(tickers='^RUI', start='2014-03-11', end='2024-07-10')
org_data

[*********************100%***********************]  1 of 1 completed


Price,Adj Close,Close,High,Low,Open,Volume
Ticker,^RUI,^RUI,^RUI,^RUI,^RUI,^RUI
Date,,,,,,
2014-03-11 00:00:00+00:00,1044.680054,1044.680054,1053.300049,1042.550049,1050.920044,0
2014-03-12 00:00:00+00:00,1045.270020,1045.270020,1045.280029,1037.119995,1042.479980,0
2014-03-13 00:00:00+00:00,1032.939941,1032.939941,1048.660034,1030.319946,1046.630005,0
2014-03-14 00:00:00+00:00,1030.800049,1030.800049,1036.599976,1029.910034,1031.880005,0
2014-03-17 00:00:00+00:00,1040.219971,1040.219971,1042.510010,1033.989990,1033.989990,0
...,...,...,...,...,...,...
2024-07-02 00:00:00+00:00,2998.479980,2998.479980,2998.820068,2972.939941,2974.409912,0
2024-07-03 00:00:00+00:00,3013.159912,3013.159912,3014.360107,2998.340088,3009.159912,0


In [198]:
data = pd.DataFrame(index=org_data.index)
data.index.name = 'Date'
data['Adj Close'] = org_data['Adj Close']['^RUI']
data['Open'] = org_data['Open']['^RUI']
data['Close'] = org_data['Close']['^RUI']
data['High'] = org_data['High']['^RUI']
data['Low'] = org_data['Low']['^RUI']
'Freq=',(data.index.max() - data.index.min())/len(data)

('Freq=', Timedelta('1 days 10:50:27.933820700'))

In [199]:
from app.helper.importer import go

fig = go.Figure()
t = data
fig.add_trace(
    go.Candlestick(x=t.index, open=t['Open'], close=t['Close'], high=t['High'], low=t['Low']))
fig.show()

In [200]:
full_date_range = pd.date_range(start=data.index.min(), end=data.index.max(), freq='D') 
full_range_df = pd.DataFrame(index=full_date_range)
no_gap_data = full_range_df.merge(data, how='left', left_index=True, right_index=True)
no_gap_data.index.name = 'Date'
no_gap_data['index'] = range(1, len(no_gap_data)+1)
no_gap_data

for column in set(no_gap_data.columns) & set(data):
    # Use `merge_asof` to get the next row where the current column is non-NA
    no_gap_data[[column]]
    data[[column]]
    no_gap_data[column] = pd.merge_asof(
        no_gap_data[[column]],  # the data to merge
        data[[column]],  # merging with rows that have no missing data
        left_index=True, right_index=True, 
        direction='forward'  # Get the next non-NA value for each row
    )[column + '_y']
no_gap_data

,Adj Close,Open,Close,High,Low,index
Date,,,,,,
2014-03-11 00:00:00+00:00,1044.680054,1050.920044,1044.680054,1053.300049,1042.550049,1
2014-03-12 00:00:00+00:00,1045.270020,1042.479980,1045.270020,1045.280029,1037.119995,2
2014-03-13 00:00:00+00:00,1032.939941,1046.630005,1032.939941,1048.660034,1030.319946,3
2014-03-14 00:00:00+00:00,1030.800049,1031.880005,1030.800049,1036.599976,1029.910034,4
2014-03-15 00:00:00+00:00,1040.219971,1033.989990,1040.219971,1042.510010,1033.989990,5
...,...,...,...,...,...,...
2024-07-05 00:00:00+00:00,3027.860107,3013.479980,3027.860107,3029.439941,3009.899902,3770
2024-07-06 00:00:00+00:00,3030.889893,3031.449951,3030.889893,3036.229980,3025.479980,3771
2024-07-07 00:00:00+00:00,3030.889893,3031.449951,3030.889893,3036.229980,3025.479980,3772


In [201]:
'Freq=',(no_gap_data.index.max() - no_gap_data.index.min())/len(no_gap_data)

('Freq=', Timedelta('0 days 23:59:37.106518282'))

In [202]:
from app.helper.importer import go

fig = go.Figure()
t = no_gap_data
fig.add_trace(
    go.Candlestick(x=t.index, open=t['Open'], close=t['Close'], high=t['High'], low=t['Low']))
fig.show()

In [203]:
position_max_time = 3 * 4 * 4 * 4 * 4 # 768 5mins = 64h = 2.66D
action_delay = 2
rolling_window = position_max_time - action_delay
n2_data = no_gap_data

In [204]:
n2_data['max_high'] = n2_data['High'].rolling(rolling_window, min_periods=rolling_window).max().shift(
    -position_max_time)
n2_data['min_low'] = n2_data['Low'].rolling(rolling_window, min_periods=rolling_window).min().shift(-position_max_time)

In [205]:
# todo: check direction of rolling window indexes as distance
n2_data['max_high_distance'] = n2_data['High'].rolling(position_max_time).apply(lambda x: x.argmax(), raw=True).shift(
    -position_max_time) + action_delay

n2_data['min_low_distance'] = n2_data['Low'].rolling(position_max_time).apply(lambda x: x.argmin(), raw=True).shift(
    -position_max_time) + action_delay

In [207]:
n2_data['next_high'] = n2_data['High'].shift(-1)
n2_data['next_low'] = n2_data['Low'].shift(-1)

In [208]:
n2_data['max_long_profit'] = n2_data['max_high'] - n2_data[['next_high', 'High']].max(axis='columns')
n2_data['max_short_profit'] = n2_data[['next_low', 'Low']].min(axis='columns') - n2_data['min_low']
n2_data['weighted_max_long_profit'] = (
        n2_data['max_long_profit'] * 0.5 + 0.5 * (
        (position_max_time - n2_data['max_high_distance']) / position_max_time))
n2_data['weighted_max_short_profit'] = (
        n2_data['max_short_profit'] * 0.5 + 0.5 * (
        (position_max_time - n2_data['min_low_distance']) / position_max_time))
n2_data

In [209]:
# t = n2_data[['High', 'Low', 'next_low', 'next_high', 'max_high_distance', 'min_low_distance']]
n2_data['long_drawdown'] = -(n2_data['High'] - n2_data['next_low']).clip(lower=0)
n2_data['long_drawdown'] = np.where(n2_data['High'].rolling(window=rolling_window).max() <= n2_data['High'], n2_data['High'], n2_data['long_drawdown'])

In [210]:
n2_data

,Adj Close,Open,Close,High,Low,index,max_high,min_low,max_high_distance,min_low_distance,...,q9_max_high,q9min_low,q9max_high_distance,q9min_low_distance,q10_max_high,q10min_low,q10max_high_distance,q10min_low_distance,next_high,next_low
Date,,,,,,,,,,,,,,,,,,,,,
2014-03-11 00:00:00+00:00,1044.680054,1050.920044,1044.680054,1053.300049,1042.550049,1,1191.589966,995.549988,436.0,703.0,...,1191.589966,995.549988,357.0,624.0,1191.589966,995.549988,434.0,701.0,1045.280029,1037.119995
2014-03-12 00:00:00+00:00,1045.270020,1042.479980,1045.270020,1045.280029,1037.119995,2,1191.589966,995.549988,435.0,702.0,...,1191.589966,995.549988,356.0,623.0,1191.589966,995.549988,433.0,700.0,1048.660034,1030.319946
2014-03-13 00:00:00+00:00,1032.939941,1046.630005,1032.939941,1048.660034,1030.319946,3,1191.589966,995.549988,434.0,701.0,...,1191.589966,995.549988,355.0,622.0,1191.589966,995.549988,432.0,699.0,1036.599976,1029.910034
2014-03-14 00:00:00+00:00,1030.800049,1031.880005,1030.800049,1036.599976,1029.910034,4,1191.589966,995.549988,433.0,700.0,...,1191.589966,995.549988,354.0,621.0,1191.589966,995.549988,431.0,698.0,1042.510010,1033.989990
2014-03-15 00:00:00+00:00,1040.219971,1033.989990,1040.219971,1042.510010,1033.989990,5,1191.589966,995.549988,432.0,699.0,...,1191.589966,995.549988,353.0,620.0,1191.589966,995.549988,430.0,697.0,1042.510010,1033.989990
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-07-05 00:00:00+00:00,3027.860107,3013.479980,3027.860107,3029.439941,3009.899902,3770,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3036.229980,3025.479980
2024-07-06 00:00:00+00:00,3030.889893,3031.449951,3030.889893,3036.229980,3025.479980,3771,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3036.229980,3025.479980
2024-07-07 00:00:00+00:00,3030.889893,3031.449951,3030.889893,3036.229980,3025.479980,3772,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3036.229980,3025.479980


In [211]:
n2_data['max_long_drawdown'] = n2_data['long_drawdown'].rolling(window=rolling_window).apply(
    lambda x: -x[:x.argmax()].clip(upper=0).sum()).shift(-position_max_time)
n2_data['count_long_drawdown'] = n2_data['long_drawdown'].rolling(window=rolling_window).apply(
    lambda x: x[:x.argmax()].clip(upper=0).count()).shift(-position_max_time)
n2_data['relative_max_long_drawdown'] = n2_data['max_long_drawdown']/n2_data['count_long_drawdown']

In [212]:
number_of_samples = 10
start = n2_data.index.min()
end = n2_data.index.max()
targets = pd.date_range(start, end, freq= (end-start)/number_of_samples)
t_df = n2_data.dropna()
samples = t_df.reset_index().sort_values(by='Date')['Date'].iloc[[i for i in range(0, len(t_df), int(len(t_df)/number_of_samples))]].to_list()
samples_df = n2_data.loc[samples]
samples_df

,Adj Close,Open,Close,High,Low,index,max_high,min_low,max_high_distance,min_low_distance,...,q9_max_high,q9min_low,q9max_high_distance,q9min_low_distance,q10_max_high,q10min_low,q10max_high_distance,q10min_low_distance,next_high,next_low
Date,,,,,,,,,,,,,,,,,,,,,
2014-03-11 00:00:00+00:00,1044.680054,1050.920044,1044.680054,1053.300049,1042.550049,1,1191.589966,995.549988,436.0,703.0,...,1191.589966,995.549988,357.0,624.0,1191.589966,995.549988,434.0,701.0,1045.280029,1037.119995
2015-01-05 00:00:00+00:00,1123.369995,1140.489990,1123.369995,1140.489990,1121.479980,301,1294.979980,995.549988,769.0,403.0,...,1294.979980,995.549988,690.0,324.0,1294.979980,995.549988,767.0,401.0,1128.329956,1107.339966
2015-11-01 00:00:00+00:00,1167.640015,1153.650024,1167.640015,1168.829956,1153.650024,601,1477.500000,995.549988,763.0,103.0,...,1477.500000,995.549988,684.0,24.0,1477.500000,995.549988,761.0,101.0,1168.829956,1153.650024
2016-08-27 00:00:00+00:00,1208.099976,1201.819946,1208.099976,1209.920044,1201.819946,901,1629.729980,1154.339966,756.0,69.0,...,1629.729980,1200.680054,677.0,2.0,1629.729980,1154.339966,754.0,67.0,1209.920044,1201.819946
2017-06-23 00:00:00+00:00,1351.390015,1348.719971,1351.390015,1352.910034,1347.040039,1201,1678.280029,1296.229980,764.0,551.0,...,1678.280029,1296.229980,685.0,472.0,1678.280029,1296.229980,762.0,549.0,1358.109985,1350.650024
2018-04-19 00:00:00+00:00,1493.449951,1501.689941,1493.449951,1501.689941,1487.469971,1501,1879.400024,1198.540039,672.0,703.0,...,1879.400024,1198.540039,593.0,624.0,1879.400024,1198.540039,670.0,701.0,1494.010010,1476.199951
2019-02-13 00:00:00+00:00,1527.810059,1523.359985,1527.810059,1532.250000,1523.359985,1801,2252.389893,1198.540039,764.0,403.0,...,2252.389893,1198.540039,685.0,324.0,2252.389893,1198.540039,762.0,401.0,1530.510010,1516.250000
2019-12-10 00:00:00+00:00,1730.939941,1733.010010,1730.939941,1735.979980,1728.229980,2101,2672.000000,1198.540039,757.0,103.0,...,2672.000000,1198.540039,678.0,24.0,2672.000000,1198.540039,755.0,101.0,1736.689941,1730.939941
2020-10-05 00:00:00+00:00,1902.719971,1870.750000,1902.719971,1902.949951,1870.750000,2401,2672.000000,1807.069946,457.0,26.0,...,2672.000000,1921.750000,378.0,660.0,2672.000000,1807.069946,455.0,24.0,1916.800049,1874.579956


In [214]:
from plotly.subplots import make_subplots

t = no_gap_data
fig = make_subplots(
    rows=3, cols=1, shared_xaxes=True,
    row_heights=[0.3, 0.7, 0.3],  # Adjust height proportions if needed
    vertical_spacing=0.01  # Spacing between the charts
)
fig.add_trace(
    go.Candlestick(x=t.index, open=t['Open'], close=t['Close'], high=t['High'], low=t['Low']), row=2, col=1)
fig.add_scatter(x=t.index, y=t['max_high'], mode='lines', line=dict(color='blue', width=1), name='max_high')
fig.add_scatter(x=t.index, y=t['min_low'], mode='lines', line=dict(color='red', width=1), name='min_low')
fig.add_scatter(x=t.index, y=t['long_drawdown'], mode='lines', line=dict(color='orange', width=1), name='long_drawdown')
fig.add_scatter(x=t.index, y=t['max_long_drawdown'], mode='lines', line=dict(color='pink', width=1),
                name='max_long_drawdown')
fig.add_scatter(x=t.index, y=t['relative_max_long_drawdown'], mode='lines', line=dict(color='brown', width=1),
                name='relative_max_long_drawdown')
fig.show()

In [143]:
no_gap_data[no_gap_data['min_low_distance'].isna()]

In [152]:
with_min_low_distance = no_gap_data[no_gap_data['min_low_distance'].notna()].index
no_gap_data.loc[with_min_low_distance,['High']]

,High
Date,
2014-03-11 00:00:00+00:00,1053.300049
2014-03-12 00:00:00+00:00,1045.280029
2014-03-13 00:00:00+00:00,1048.660034
2014-03-14 00:00:00+00:00,1036.599976
2014-03-15 00:00:00+00:00,1042.510010
...,...
2022-05-29 00:00:00+00:00,2288.409912
2022-05-30 00:00:00+00:00,2288.409912
2022-05-31 00:00:00+00:00,2288.409912


In [169]:
with_min_low_distance = no_gap_data[no_gap_data['min_low_distance'].notna()].index
no_gap_data.loc[with_min_low_distance,'High']
no_gap_data.loc[with_min_low_distance, 'min_low_distance'].astype(int)

Date
2014-03-11 00:00:00+00:00    703
2014-03-12 00:00:00+00:00    702
2014-03-13 00:00:00+00:00    701
2014-03-14 00:00:00+00:00    700
2014-03-15 00:00:00+00:00    699
                            ... 
2022-05-29 00:00:00+00:00    138
2022-05-30 00:00:00+00:00    137
2022-05-31 00:00:00+00:00    136
2022-06-01 00:00:00+00:00    135
2022-06-02 00:00:00+00:00    134
Freq: D, Name: min_low_distance, Length: 3006, dtype: int64

In [177]:
with_min_low_distance = no_gap_data[no_gap_data['min_low_distance'].notna()].index
t = no_gap_data.loc[with_min_low_distance, ['min_low_distance']].astype(int)
t[t['min_low_distance'].isna()]
no_gap_data.loc[with_min_low_distance, 'min_low_distance'].astype(int)

Date
2014-03-11 00:00:00+00:00    703
2014-03-12 00:00:00+00:00    702
2014-03-13 00:00:00+00:00    701
2014-03-14 00:00:00+00:00    700
2014-03-15 00:00:00+00:00    699
                            ... 
2022-05-29 00:00:00+00:00    138
2022-05-30 00:00:00+00:00    137
2022-05-31 00:00:00+00:00    136
2022-06-01 00:00:00+00:00    135
2022-06-02 00:00:00+00:00    134
Freq: D, Name: min_low_distance, Length: 3006, dtype: int64

In [172]:
with_min_low_distance = no_gap_data[no_gap_data['min_low_distance'].notna()].index
no_gap_data.loc[with_min_low_distance,'High']
no_gap_data.loc[with_min_low_distance, 'min_low_distance'].astype(int)
no_gap_data.loc[with_min_low_distance, 'max_short_drawdown'] = no_gap_data.loc[with_min_low_distance,'High'].rolling(
    window=no_gap_data.loc[with_min_low_distance, 'min_low_distance'].astype(int), 
    on=with_min_low_distance
).max()

ValueError: window must be an integer 0 or greater